<a href="https://colab.research.google.com/github/dan-candeira/Nuforc-Project/blob/master/5.7-Preparacao/5.7-nuforc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
! pip install pandasql
import pandasql as ps

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dan-candeira/Nuforc-Project/master/5.2-Coleta/ufoCollection.csv')\
.drop(columns = ['Unnamed: 0'])

In [3]:
df.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01


In [4]:
query = '''
  SELECT * FROM df
  WHERE State != ''
  AND LOWER(State) != 'none'
  AND LOWER(Shape) != 'unknown'
  AND Shape != ''
  AND City != ''
  AND LOWER(City) != 'none'
  '''
clean_df = ps.sqldf(query, locals())

In [5]:
clean_df.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01


In [6]:
us_states = pd.read_csv('https://raw.githubusercontent.com/dan-candeira/Nuforc-Project/master/5.7-Preparacao/states.csv', sep=';')

In [7]:
us_states.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [8]:
query = '''
  SELECT clean_df.* FROM clean_df, us_states
  WHERE clean_df.State = Abbreviation
'''
us_df = ps.sqldf(query, locals())

In [9]:
us_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80174 entries, 0 to 80173
Data columns (total 7 columns):
Date / Time    80174 non-null object
City           80174 non-null object
State          80174 non-null object
Shape          80174 non-null object
Duration       78279 non-null object
Summary        80165 non-null object
Posted         80172 non-null object
dtypes: object(7)
memory usage: 4.3+ MB


In [10]:
new_df = us_df.drop(columns=['Posted', 'Duration', 'Summary'])

In [11]:
new_df.head()

,Date / Time,City,State,Shape
0,9/30/97 22:00,Madison,WI,Light
1,9/28/97 23:15,San Francisco,CA,Triangle
2,9/27/97 23:00,Egan,SD,Other
3,9/27/97 05:00,Crestwood,KY,Disk
4,9/25/97 22:00,Clearfield,UT,Triangle


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80174 entries, 0 to 80173
Data columns (total 4 columns):
Date / Time    80174 non-null object
City           80174 non-null object
State          80174 non-null object
Shape          80174 non-null object
dtypes: object(4)
memory usage: 2.4+ MB


In [13]:
query = '''
  SELECT Shape, COUNT(Shape) as views FROM new_df
  GROUP BY Shape
  HAVING views > 1000
'''

popular_shapes = ps.sqldf(query, locals())

In [14]:
popular_shapes.head()

,Shape,views
0,Changing,2275
1,Chevron,1041
2,Cigar,1896
3,Circle,9315
4,Cylinder,1370


In [ ]:
query = '''
  SELECT new_df.* FROM new_df, popular_shapes
  WHERE new_df.Shape = popular_shapes.Shape
'''
final_df = ps.sqldf(query, locals())

In [ ]:
final_df.info()

In [ ]:
final_df.to_csv('df_OVNI_limpo.csv')